# Edit Distance主要有三种情况：
（1）插入：造成长度增加 1
（2）删除：造成长度减小 1
（3）替换：不对长度产生影响，但是对内容产生影响

## string1 = A B D
## string2 = A B C D

如果string2删除了一个字符，string1就等于string2了，那就是代表string1 insert operation eaquals string2 

那么怎么删除这个字符，或者删除哪个字符能让他俩相等，这个需要需要进行判断。

In [ ]:
下面的方法并没有采用这种方法，而是采取了

In [1]:
from functools import lru_cache

In [20]:
solution = {}

lru_cache的原理：https://www.cnblogs.com/muzinan110/p/11158608.html

In [68]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [69]:
edit_distance('ABC', 'ABD')

1

In [70]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABD'): 'ADD D',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABD'): 'ADD D',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABD'): 'SUB C => D'}

In [44]:
solution = {}

In [71]:
edit_distance('ACB','AB')

1

In [72]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABD'): 'ADD D',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABD'): 'ADD D',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABD'): 'SUB C => D',
 ('AC', 'A'): 'DEL C',
 ('AC', 'AB'): 'SUB C => B',
 ('ACB', 'A'): 'DEL B',
 ('ACB', 'AB'): ''}

In [73]:
solution ={}

In [74]:
edit_distance('BDRF','AECSD')

5

In [75]:
solution

{('B', 'A'): 'SUB B => A',
 ('B', 'AE'): 'ADD E',
 ('B', 'AEC'): 'ADD C',
 ('B', 'AECS'): 'ADD S',
 ('B', 'AECSD'): 'ADD D',
 ('BD', 'A'): 'DEL D',
 ('BD', 'AE'): 'SUB D => E',
 ('BD', 'AEC'): 'ADD C',
 ('BD', 'AECS'): 'ADD S',
 ('BD', 'AECSD'): '',
 ('BDR', 'A'): 'DEL R',
 ('BDR', 'AE'): 'DEL R',
 ('BDR', 'AEC'): 'SUB R => C',
 ('BDR', 'AECS'): 'ADD S',
 ('BDR', 'AECSD'): 'DEL R',
 ('BDRF', 'A'): 'DEL F',
 ('BDRF', 'AE'): 'DEL F',
 ('BDRF', 'AEC'): 'DEL F',
 ('BDRF', 'AECS'): 'SUB F => S',
 ('BDRF', 'AECSD'): 'ADD D'}

In [77]:
solution[('BD','AE')]

'SUB D => E'

In [139]:
from collections import defaultdict
transfer = list()

In [140]:
def edit_trnasfer(str1, str2):
    # 1 基线条件是 如果str1和str2的长度都为1，那么返回当前的操作
    transfer.append(solution[(str1, str2)]) 
    if len(str1) == 1: return solution[(str1, str2)]
    if len(str2) == 1: return solution[(str1, str2)]
    if len(str1) >= len(str2):
        return edit_trnasfer(str1[:-1], str2[:len(str1[:-1])])
    else:
        return edit_trnasfer(str1[:len(str2[:-1])], str2[:-1])
    #transfer.append()
    

In [141]:
edit_trnasfer('BDRF', 'AECSD')

'SUB B => A'

In [169]:
def trans_process(str1, str2, trans):
    return ("{}" + ' through ' + str(trans[::-1]) + ' as '+"{}").format(str1, str2)

In [170]:
trans_process('BDRF','AECSD',transfer)

"BDRF through ['SUB B => A', 'SUB D => E', 'SUB R => C', 'SUB F => S', 'ADD D'] as AECSD"

# 通过transfer这个过程 BDRF 就可以变成 AECSD。